## XGBoost single instrument - New input

This code uses the new input shape, using the librosa fourier and inverse fourier transform to generate the spectrograms from waveform

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

# Import external functions from the functions folder
import sys
sys.path.append('../../functions')
import functions as f

In [2]:
# Set local path to the folder containing the .wav audio files
path = 'C:/Users/lucvo/VScode/Machine_learning/Audio_data/nsynth-valid.jsonwav/nsynth-valid/audio/'

In [3]:
# Create a training set of spectrograms and labels
X, y, label_dict = f.create_single_inst_classification_set_new_input(N_samples=5000, classes=['keyboard','bass','flute', 'guitar'], path=path)

# Split the data into training, validation and test sets
X_train, X_val, X_test, y_train, y_val, y_test = f.split_data(X, y, val_frac=0.1, test_frac=0.1)

The dataset might be unbalanced


In [4]:
print(X_train.shape)
print(y_train.shape)
print(label_dict)

(4000, 129150)
(4000,)
{0: 'keyboard', 1: 'bass', 2: 'flute', 3: 'guitar'}


In [7]:
import xgboost as xgb
from sklearn.metrics import confusion_matrix

In [5]:
# Use only the first 2000 samples for training
X_train = X_train[:2000]
y_train = y_train[:2000]

In [8]:
# Use an XGBoost classifier
model = xgb.XGBClassifier(objective='multi:softmax', num_class=len(label_dict), n_estimators=100, max_depth=3, learning_rate=0.1, n_jobs=-1)

# Train the model
eval_set = [(X_train, y_train), (X_val, y_val)]
model.fit(X_train, y_train, eval_set=eval_set, eval_metric='merror', early_stopping_rounds=10, verbose=True)

# Predict the labels of the validation set
y_pred = model.predict(X_val)

# Print the accuracy
accuracy = np.sum(y_pred == y_val) / len(y_val)
print('Accuracy:', accuracy)

c:\Users\lucvo\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\lucvo\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


In [ ]:
# Plot the training and validation error as a function of the number of trees
results = model.evals_result()
train_error = results['validation_0']['merror']
val_error = results['validation_1']['merror']
n_trees = range(len(train_error))
plt.plot(n_trees, train_error, label='Train error')
plt.plot(n_trees, val_error, label='Validation error')
plt.xlabel('Number of trees')
plt.ylabel('Classification error')
plt.legend()
plt.show()

In [ ]:
# Plot the confusion matrix
conf_matrix = confusion_matrix(y_val, y_pred)
conf_normed = conf_matrix / np.sum(conf_matrix, axis=1)[:, np.newaxis]
plt.imshow(conf_normed, cmap='Blues')
plt.colorbar()
plt.xlabel('Predicted')
plt.ylabel('True')
plt.xticks(range(len(label_dict)), label_dict.keys(), rotation=45)
plt.yticks(range(len(label_dict)), label_dict.keys())
plt.show()